In [24]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from transformers import Trainer, TrainingArguments
import pandas as pd
import wandb

# def tokenization(x):
#     return tokenizer(x, padding = 'max_length', truncation=True, max_length = 512, return_tensors="pt")

In [25]:
train_data = pd.read_csv('data\si630w22-hw3-train.csv')
dev_data = pd.read_csv('data\si630w22-hw3-dev.csv')
# test_data = pd.read_csv('data\si630w22-hw3-test.public.csv')
q_and_a_data = pd.read_csv('data\si630w22-hw3-data.csv')

In [26]:
train_data.rename(columns={'id':'question_id'}, inplace = True)
dev_data.rename(columns={'id':'question_id'}, inplace = True)
# test_data.rename(columns={'id':'question_id'}, inplace = True)
q_and_a_data['combined'] = q_and_a_data.question_text + '[SEP]' + q_and_a_data.reply_text

merged_train_data = pd.merge(q_and_a_data, train_data, on='question_id')
merged_train_data.to_csv('data\merged_train_data.csv', index=False)

merged_dev_data = pd.merge(q_and_a_data, dev_data, on='question_id')
merged_dev_data.to_csv('data\merged_dev_data.csv', index=False)

# merged_test_data = pd.merge(q_and_a_data, test_data, on='question_id')
# merged_test_data.to_csv('data\merged_test_data.csv', index=False)

In [28]:
data_files = {"train": "data\merged_train_data.csv",
              "dev": "data\merged_dev_data.csv",}
#               "test": "data\merged_test_data.csv"}
dataset = load_dataset("csv", data_files=data_files)
dataset

Using custom data configuration default-5ea3da82c98e36c5


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\User\.cache\huggingface\datasets\csv\default-5ea3da82c98e36c5\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question_id', 'question_text', 'reply_id', 'reply_text', 'rlen', 'combined', 'annotator_id', 'rating', 'group'],
        num_rows: 17845
    })
    dev: Dataset({
        features: ['question_id', 'question_text', 'reply_id', 'reply_text', 'rlen', 'combined', 'annotator_id', 'rating', 'group'],
        num_rows: 3810
    })
})

In [31]:
model = AutoModel.from_pretrained("microsoft/MiniLM-L12-H384-uncased")
tokenizer = AutoTokenizer.from_pretrained("microsoft/MiniLM-L12-H384-uncased")

training_args = TrainingArguments(
    "test-trainer",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=dataset['train'],
#    eval_dataset=dataset['dev'],
#     data_collator=data_collator,
    tokenizer = tokenizer,
#     compute_metrics=compute_metrics,
)

loading configuration file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/config.json from cache at C:\Users\User/.cache\huggingface\transformers\ceb753d3f27a8c0d09184f35884666cda91b8ae610cd2a54d89793ac7663f1f9.13815020fd994b27db9974c0ce0ec4c47dfac6c8f11bf1a35a0a06d5b165665a
Model config BertConfig {
  "_name_or_path": "microsoft/MiniLM-L12-H384-uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/pytorch_model.bi

In [32]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertModel.forward` and have been ignored: question_id, rating, group, reply_id, question_text, reply_text, annotator_id, rlen, combined. If question_id, rating, group, reply_id, question_text, reply_text, annotator_id, rlen, combined are not expected by `BertModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 0
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6693
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


IndexError: Invalid key: 13979 is out of bounds for size 0